In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2

from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


In [ ]:
def get_detected_traffic_lights(boxes, scores, classes):
    inds = ((classes == 10) & (scores >= 0.15)).nonzero()
    
    filtered_boxes = boxes[inds[0], ...]
    filtered_scores = scores[inds[0], ...]
    filtered_classes = classes[inds[0], ...]
    return filtered_boxes, filtered_scores, filtered_classes
        
def crop_detection(image, boxes, path, img):
        height = image.shape[1]
        width = image.shape[2]
        
        # Box coordinates are scaled and need to be converted
        # into image coordinates
        boxes[:, 0] *= height
        boxes[:, 1] *= width
        boxes[:, 2] *= height
        boxes[:, 3] *= width
        
        # cropping needs integers 
        boxes = boxes.astype(int)
        
        # only needed for testing
        name = img.split('\\')[1]
        name = name.split('.')[0]
        
        # loop over every detected traffic light
        for i in range(len(boxes)):
            bot, left, top, right = boxes[i,...]

            cropped = image[0,bot:top,left:right]
            
            # only needed for testing
            save_img = cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR)
            cv2.imwrite(path+'/'+name+'_'+str(i)+'_cropped.jpg',save_img)
            #plt.imshow(cropped)

In [ ]:
def load_SSD_MobileNet(path_frozen_model):
    #loads a frozen SSD MobileNet
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(path_frozen_model, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
            
        # get Tensors for input image + boxes for detected objects + classification for each box
        # + scores for how certain the classifier is
        image_tensor = graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = graph.get_tensor_by_name('detection_boxes:0')
        detection_classes = graph.get_tensor_by_name('detection_classes:0')
        detection_scores = graph.get_tensor_by_name('detection_scores:0')       
    return graph, image_tensor, detection_boxes,detection_scores,detection_classes

In [ ]:
def detect_objects_in_single_img(detection_graph, image_tensor, detection_boxes,\
                                detection_scores, detection_classes, image):
  
    with tf.Session(graph=detection_graph) as sess:                  
        (boxes, scores, classes) = sess.run([detection_boxes, detection_scores, detection_classes], 
                                            feed_dict={image_tensor: image})

        # Remove unnecessary dimensions
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes)

        return boxes, scores, classes


In [ ]:
frozen_model='frozen_inference_graph.pb'
test_images= glob.glob('testing/*.jpg')
path_cropped_images = 'testing/cropped'

In [ ]:
detection_graph,image_tensor,detection_boxes,detection_scores,detection_classes = load_SSD_MobileNet(frozen_model)
for img in test_images[0:2]:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.expand_dims(image, 0)
    boxes,scores,classes = detect_objects_in_single_img(detection_graph, image_tensor, detection_boxes,\
                                                        detection_scores, detection_classes, image)
    
    # Filter boxes with a confidence score less than `confidence_cutoff`
    boxes, scores, classes = get_detected_traffic_lights(boxes, scores, classes)
    crop_detection(image, boxes,path_cropped_images, img)

In [ ]:
cropped_images_red = glob.glob('cropped_sorted_training/red/*.jpg')
cropped_images_yellow = glob.glob('cropped_sorted_training/yellow/*.jpg')
cropped_images_green = glob.glob('cropped_sorted_training/green/*.jpg')
cropped_images_false = glob.glob('cropped_sorted_training/false/*.jpg')

labels = []
x_data = []

for img in cropped_images_red:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(64,64))
    x_data.append(image)
    labels.append(0)
    
for img in cropped_images_green:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(64,64))
    x_data.append(image)
    labels.append(2)
    
for img in cropped_images_yellow:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(64,64))
    x_data.append(image)
    labels.append(1)
    
for img in cropped_images_false:
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(64,64))
    x_data.append(image)
    labels.append(4)


x_data = np.asarray(x_data)
one_hot_labels = to_categorical(labels, num_classes=5)

x_train, label_train = shuffle(x_data, one_hot_labels)

In [ ]:
model = Sequential()

model.add(Lambda(lambda x: x/255 - 0.5, input_shape=(64,64,3)))
model.add(Conv2D(6,(5,5),strides=(2, 2), padding='same',activation='relu',\
          use_bias=True,kernel_initializer='TruncatedNormal', bias_initializer='zeros'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))

model.add(Conv2D(12,(3,3),strides=(2, 2), padding='same',activation='relu',\
          use_bias=True,kernel_initializer='TruncatedNormal', bias_initializer='zeros'))

model.add(Dropout(0.25))

model.add(Conv2D(18,(1,1),strides=(1, 1), padding='valid',activation='relu',\
          use_bias=True,kernel_initializer='TruncatedNormal', bias_initializer='zeros'))

model.add(Flatten())

model.add(Dense(100,activation='relu',kernel_initializer='TruncatedNormal',\
                bias_initializer='zeros'))

model.add(Dropout(0.5))

model.add(Dense(10,activation='relu',kernel_initializer='TruncatedNormal',\
                bias_initializer='zeros'))

model.add(Dense(5,activation='relu',kernel_initializer='TruncatedNormal',\
                bias_initializer='zeros'))

optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)

model.fit(x=x_train, y=label_train, epochs=5, batch_size=3)

In [ ]:
model.save('model.h5')